# Upper Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ x := U x $ where $ U $ is upper triangular.  Vector $ y $ is not to be touched (and, indeed, not even passed into the routines).  This is a little trickier than you might think, especially when you work with lower triangular matrix $ L $, later.  Indeed, you may want to do a few small problems by hand if you don't get the right answer.  Also, PictureFLAME may help you see what is going on.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm that takes dot products

<img src="../images/trmvp_un_dot.png" alt="Matrix-matrix multiplication by columns picture" width="80%">

## The `Trmvp_un_unb_var1!( U, x )` routine

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := U x $.  The "_un_" in the name of the routine indicates this is the "upper, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots!( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.scal!( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [30]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trmvp_un_unb_var1!(U, x)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    while size(UTL, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.scal!( upsilon11, chi1 )
        laff.dots!( u12t, x2, chi1 )
        
        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

    end

    flame.merge_2x1!(xT, 
                     xB, x)

end

Trmvp_un_unb_var1! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [31]:
U = rand(4, 4)
x = rand(4)
xold = rand(4)

# Notice that U is not upper triangular.  We will only use the upper triangular part.

println( "U before =" )
U

U before =


4×4 Array{Float64,2}:
 0.568712  0.0484043  0.239765  0.597326
 0.513997  0.451848   0.109589  0.94517 
 0.502999  0.229655   0.999115  0.150114
 0.601896  0.674244   0.70824   0.93796 

In [32]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.8922830772807757 
 0.19250286062510757
 0.31566567331804163
 0.11048724269184662

In [33]:
laff.copy!( x, xold )   # save the original vector x

Trmvp_un_unb_var1!( U, x )

println( "after =" )
x

after =


4-element Array{Float64,1}:
 0.6584528290959286 
 0.22600478716218547
 0.33197207995421807
 0.10363262985771861

In [34]:
using LinearAlgebra
UpperTriangular( U ) * xold

4-element Array{Float64,1}:
 0.6584528290959286 
 0.22600478716218547
 0.33197207995421807
 0.10363262985771861

In [35]:
println( "x - ( UpperTriangular( U ) * xold ) = " ) # UpperTriangular() makes the matrix upper triangular
x - ( UpperTriangular( U ) * xold )

x - ( UpperTriangular( U ) * xold ) = 


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

<img src="../images/trmvp_un_axpy.png" alt="Matrix-matrix multiplication by columns picture" width="80%">

## The `Trmvp_un_unb_var2!( U, x )` routine

This routine, given upper triangular $ U \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := U x $.  The "_un_" in the name of the routine indicates this is the "upper triangular, no transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
<li> <code> laff.scal!( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [36]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Trmvp_un_unb_var2!(U, x)

    UTL, UTR, 
    UBL, UBR  = flame.part_2x2(U, 
                               0, 0, "TL")

    xT, 
    xB  = flame.part_2x1(x, 
                         0, "TOP")

    while size(UTL, 1) < size(U, 1)

        U00,  u01,       U02,  
        u10t, upsilon11, u12t, 
        U20,  u21,       U22   = flame.repart_2x2_to_3x3(UTL, UTR, 
                                                         UBL, UBR, 
                                                         1, 1, "BR")

        x0,   
        chi1, 
        x2    = flame.repart_2x1_to_3x1(xT, 
                                        xB, 
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( chi1, u01, x0 )
        laff.scal!( upsilon11, chi1 )
        
        #------------------------------------------------------------#

        UTL, UTR, 
        UBL, UBR  = flame.cont_with_3x3_to_2x2(U00,  u01,       U02,  
                                               u10t, upsilon11, u12t, 
                                               U20,  u21,       U22,  
                                               "TL")

        xT, 
        xB  = flame.cont_with_3x1_to_2x1(x0,   
                                         chi1, 
                                         x2,   
                                         "TOP")

    end

    flame.merge_2x1!(xT, 
                     xB, x)

end

Trmvp_un_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [37]:
U = rand(4, 4)
x = rand(4)
xold = rand(4)

# Notice that U is not upper triangular.  We will only use the upper triangular part.

println( "U before =" )
U

U before =


4×4 Array{Float64,2}:
 0.407676   0.213559   0.954064  0.346649 
 0.75617    0.559332   0.77835   0.0199678
 0.0778386  0.705902   0.856714  0.730193 
 0.498111   0.0277845  0.281855  0.103336 

In [38]:
println( "x before =" )
x

x before =


4-element Array{Float64,1}:
 0.5959365771819232  
 0.9481134857472329  
 0.8802097318036222  
 0.026284186562488765

In [39]:
laff.copy!( x, xold )   # save the original vector x

Trmvp_un_unb_var2!( U, x )

println( "after =" )
x

after =


4-element Array{Float64,1}:
 1.294315158712078   
 1.215945841422403   
 0.77328093503125    
 0.002716094694956955

In [40]:
using LinearAlgebra
UpperTriangular( U ) * xold

4-element Array{Float64,1}:
 1.294315158712078   
 1.215945841422403   
 0.77328093503125    
 0.002716094694956955

In [41]:
println( "x - ( UpperTriangular( U ) * xold ) = " ) # UpperTriangular() makes the matrix upper triangular
x - ( UpperTriangular( U ) * xold )

x - ( UpperTriangular( U ) * xold ) = 


4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.